In [7]:
import pandas as pd

In [8]:
CustDist_WHCentral = pd.read_csv("CustDist_WHCentral.csv", sep=",", decimal=".")
del CustDist_WHCentral["Distances between Customers and Warehouse"]
CustDist_WHCentral_list = CustDist_WHCentral.values.tolist()

CustDist_WHCorner = pd.read_csv("CustDist_WHCorner.csv", sep=",", decimal=".")
del CustDist_WHCorner["Distances between Customers and Warehouse"]
CustDist_WHCorner_list = CustDist_WHCorner.values.tolist()

CustOrd = pd.read_csv("CustOrd.csv", sep=",", decimal=".")
del CustOrd["Customer"]
CustOrd_list = CustOrd.values.tolist()

In [9]:
#just to test 
import random

#creates the test list
test_list = list(range(1, 51))

#shuffles the list
for i in range(1000):
    index1 = random.randint(0,49)
    index2 = random.randint(0,49)
        
    val1 = test_list[index1]
    test_list[index1] = test_list[index2]
    test_list[index2] = val1

    
print(test_list)

[18, 25, 15, 33, 28, 21, 1, 46, 3, 27, 45, 23, 48, 43, 16, 7, 24, 11, 36, 35, 37, 13, 39, 22, 12, 42, 2, 50, 34, 41, 49, 47, 44, 20, 9, 38, 30, 4, 10, 14, 26, 40, 29, 17, 31, 32, 5, 19, 8, 6]


In [76]:
def evaluatecost(individual):
    total_distance = 0 
    load = 1000 
    
    for i in individual:
        #doesnt have enough load 
        if(load < CustOrd_list[i][0]):
            #last location -> central
            total_distance = total_distance + CustDist_WHCentral_list[0][i-1]
            load = 1000
            
            #central -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][i]
            load = load - CustOrd_list[i][0]
        
        #last client
        if(i == (len(individual) - 1)):
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[i][i-1]
            load = load - CustOrd_list[i][0]
            #next location -> warehouse
            total_distance = total_distance + CustDist_WHCentral_list[i][0]
        
        #first client
        if(i == 0):
            #warehouse -> next location
            total_distance = total_distance + CustDist_WHCentral_list[0][i]
            load = load - CustOrd_list[i][0]
        
        #has enough load 
        else: 
            #last location -> next location
            total_distance = total_distance + CustDist_WHCentral_list[i][i-1]
            load = load - CustOrd_list[i][0]
            
    return total_distance,

In [77]:
evaluatecost(test_list)



(2846,)

In [86]:
import random

import random
import numpy as np
from deap import base, creator, tools


def P1(Number_Customers, Population_Size, Number_of_Evaluations, seed):
    
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    ## permutation setup for individual,
    toolbox.register("indices", \
                     random.sample, \
                     range(Number_Customers), 
                     Number_Customers)

    toolbox.register("individual", \
                     tools.initIterate, \
                     creator.Individual, toolbox.indices)

    ## population setup,
    toolbox.register("population", \
                     tools.initRepeat, \
                     list, toolbox.individual)


    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
    toolbox.register("evaluate", evaluatecost)
    toolbox.register("select", tools.selTournament, tournsize=10)
    
    
    random.seed(seed)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=Population_Size)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.8, 0.1
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while g < Number_of_Evaluations/ Population_Size:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
    return 

In [87]:
P1(50, 100, 10000, 53)

Start of evolution
  Evaluated 100 individuals
-- Generation 1 --
  Evaluated 86 individuals
  Min 2741.0
  Max 2831.0
  Avg 2783.37
  Std 21.97391863097888
-- Generation 2 --
  Evaluated 79 individuals
  Min 2725.0
  Max 2829.0
  Avg 2773.18
  Std 26.94415706606119
-- Generation 3 --
  Evaluated 83 individuals
  Min 2725.0
  Max 2830.0
  Avg 2770.11
  Std 31.624324498706205
-- Generation 4 --
  Evaluated 82 individuals
  Min 2725.0
  Max 2827.0
  Avg 2755.94
  Std 31.71965321372951
-- Generation 5 --
  Evaluated 89 individuals
  Min 2725.0
  Max 2845.0
  Avg 2741.0
  Std 28.80277764382504
-- Generation 6 --
  Evaluated 84 individuals
  Min 2725.0
  Max 2819.0
  Avg 2742.9
  Std 30.39720381877622
-- Generation 7 --
  Evaluated 80 individuals
  Min 2725.0
  Max 2823.0
  Avg 2739.11
  Std 24.29234241482329
-- Generation 8 --
  Evaluated 65 individuals
  Min 2725.0
  Max 2810.0
  Avg 2734.61
  Std 20.12108098485879
-- Generation 9 --
  Evaluated 92 individuals
  Min 2725.0
  Max 2833.0
  

  Evaluated 81 individuals
  Min 2725.0
  Max 2809.0
  Avg 2735.61
  Std 19.42879049244594
-- Generation 79 --
  Evaluated 73 individuals
  Min 2725.0
  Max 2823.0
  Avg 2737.37
  Std 20.94882096923072
-- Generation 80 --
  Evaluated 75 individuals
  Min 2725.0
  Max 2806.0
  Avg 2738.96
  Std 21.041349766594053
-- Generation 81 --
  Evaluated 81 individuals
  Min 2725.0
  Max 2806.0
  Avg 2737.87
  Std 20.945001790420825
-- Generation 82 --
  Evaluated 78 individuals
  Min 2725.0
  Max 2807.0
  Avg 2740.79
  Std 23.242760163112326
-- Generation 83 --
  Evaluated 75 individuals
  Min 2720.0
  Max 2806.0
  Avg 2740.13
  Std 24.080139119179066
-- Generation 84 --
  Evaluated 76 individuals
  Min 2720.0
  Max 2823.0
  Avg 2734.97
  Std 22.04470684770537
-- Generation 85 --
  Evaluated 89 individuals
  Min 2720.0
  Max 2813.0
  Avg 2739.41
  Std 27.522389067828847
-- Generation 86 --
  Evaluated 89 individuals
  Min 2720.0
  Max 2823.0
  Avg 2734.62
  Std 26.084393801683945
-- Generation 8